<a href="https://colab.research.google.com/github/jcv444/FAQ/blob/master/YouTube_Whisper_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're looking at this on GitHub and new to Python Notebooks or Colab, click the Google Colab badge above 👆


#**Creating YouTube transcripts with OpenAI's Whisper model**

📺 Getting started video: https://youtu.be/kENRf82_RQs

*Colab beginner notes:*
<br>
1. These files are being loaded on a virtual machine in the cloud. Nothing is being downloaded to your computer (except for the transcript when you click to download it.) When you close this session the instance will be erased.
<br>
2. The run button is visible when you move your mouse close to the left edge of the code block. It looks kind of like this: ▶️ ...but round...and white on black...so nothing like this. You'll know it when you see it.

###**Note: For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*


**Step 1.** Follow the instructions in each block and select the options you want
<br>
**Step 2.** Get the url of the video you want to transcribe
<br>
**Step 3.** Refresh the folder on the left and download your transcript
<br>
**Step 4.** Go to your YouTube account and upload the transcript to the video it came from and use "autosync."

That's it!

Have a question? Hit me up on Twitter:[ @AndrewMayne](https://twitter.com/andrewmayne)

<br>



---


**What is this?**
<br>
This is a Python notebook that creates a transcript from a YouTube url using OpenAI's Whisper transcription model that you can then upload to YouTube using the autosync feature to create captions.
<br>  
**What is OpenAI's Whisper model?**
<br>
Whisper is an automatic speech recognition (ASR) neural net created by OpenAI that transcribes audio at close to human level.
<br>
<br>
**Why use this?**
<br>
The quality of the OpenAI Whisper model is amazing (I am slightly biased, but seriously, check it out.) You can also use it to transcribe in other languages.
<br>
<br>
**What do the different model sizes do?**
<br>
Each model size has an improvement in quality – especially with different languages. I've found that for a YouTube video with clear speech, the base model works really well. If you see transcription errors, you can try a larger model.
<br>
<br>
**Do I need timestamps?**
<br>
Nope. YouTube's autosync function will match the text to the spoken words and syncs up really well. All you need is each spoken sentence in a .txt file.
<br>
<br>
**How do I do this?**
<br>
Just follow each step. If you've never used Colab of a Python notebook, don't panic. It's super easy and runs in the cloud.
<br>
<br>
**Does this cost anything to use?**
<br>
Nope. You can use Colab for free and Whisper is an open source model.
<br>
<br>
[Tips for creating a YouTube transcript file](https://support.google.com/youtube/answer/2734799?hl=en)
<br>
[Information on OpenAI's Whisper model](https://openai.com/blog/whisper/)
<br>
[OpenAI's Whisper GitHub page](https://github.com/openai/whisper)
<br>












In [4]:
"""
1. Click the start button in the upper left side of this block to load the necessary libraries

You will need to run this every time you reload this notebook.
"""

!pip uninstall -y youtube_dl
!pip install yt-dlp
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install librosa

import whisper
import time
import librosa
import re
import yt_dlp

Found existing installation: youtube-dl 2021.12.17
Uninstalling youtube-dl-2021.12.17:
  Successfully uninstalled youtube-dl-2021.12.17
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.5 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-j_qmzxc6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-j_qmzxc6
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu 

In [5]:
"""
2. Select the model you want to use.

Base works really well so it's the default.

(For multilingual, remove ".en" from the model name.)

Click the run button after you've made your choice (or left it at default.)
"""

# model = whisper.load_model("tiny.en")
model = whisper.load_model("base.en")
# model = whisper.load_model("small.en")
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 91.8MiB/s]


In [6]:
import yt_dlp
"""
3. Click the run button and input your YouTube URL in the box below then click enter.

You can use this one to test: https://www.youtube.com/watch?v=kENRf82_RQs

The video will be loaded and the audio extracted (this is usually the longest part of the process.)

Your transcript will appear in the folder on the left (you may have to refresh the folder to see it.)

You can download the file when it's completed and upload it on your video's detail page using "autosync."
"""

# This will prompt you for a YouTube video URL
url = input("Enter a YouTube video URL: ")

# Create a yt-dlp options dictionary
ydl_opts = {
    # Specify the format as bestaudio/best
    'format': 'bestaudio/best',
    # Specify the post-processor as ffmpeg to extract audio and convert to mp3
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    # Specify the output filename as the video title
    'outtmpl': '%(title)s.%(ext)s',
}

# Download the video and extract the audio
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=True)
    file_path = ydl.prepare_filename(info_dict)

# Get the path of the file
file_path = file_path.replace('.webm', '.mp3')
file_path = file_path.replace('.m4a', '.mp3')

# Get the duration
duration = librosa.get_duration(filename=file_path)
start = time.time()
result = model.transcribe(file_path)
end = time.time()
seconds = end - start

print("Video length:", duration, "seconds")
print("Transcription time:", seconds)

# Split result["text"]  on !,? and . , but save the punctuation
sentences = re.split("([!?.])", result["text"])

# Join the punctuation back to the sentences
sentences = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
text = "\n\n".join(sentences)
for s in sentences:
  print(s)

# Save the file as .txt
name = "".join(file_path) + ".txt"
with open(name, "w") as f:
  f.write(text)

print("\n\n", "-"*100, "\n\nYour transcript is here:", name)

Enter a YouTube video URL: https://www.youtube.com/watch?v=kENRf82_RQs
[youtube] Extracting URL: https://www.youtube.com/watch?v=kENRf82_RQs
[youtube] kENRf82_RQs: Downloading webpage


[youtube] kENRf82_RQs: Downloading android vr player API JSON
[info] kENRf82_RQs: Downloading 1 format(s): 251
[download] Destination: Transcribe YouTube videos for free with OpenAI's Whisper.webm
[download] 100% of    5.19MiB in 00:00:00 at 5.83MiB/s   
[ExtractAudio] Destination: Transcribe YouTube videos for free with OpenAI's Whisper.mp3
Deleting original file Transcribe YouTube videos for free with OpenAI's Whisper.webm (pass -k to keep)


/tmp/ipython-input-2652798481.py:41: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


Video length: 368.9769791666667 seconds
Transcription time: 18.651933908462524
 Hi, I'm Andrew and this is a quick tutorial on how to use opening eyes whisper model in Google Colab to make really high quality transcriptions and captions for your YouTube videos.
 We're starting off here on the GitHub page, which is what links to the Colab notebook.
 And if you're not familiar with Colab or Jupiter or Python notebooks, any other things I mentioned, don't worry, this will be a very easy tutorial and I'll explain what you need to know as we go along.
 So you can start off here by seeing this is a copy of the notebook, but we'll just click on that badge, which will take us to the actual notebook running on Google Colab.
 So what is Google Colab?
 It's a cloud service that lets you run a Python notebook.
 What's a Python notebook?
 A Python notebook is an environment where you can run both Python code and have text that explains what's going on.
 So I have a text block here describing the no